In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# **DREAMBOOTH** 🤖

Dreambooth es un modelo de generación de aprendizaje profundo, y que fue desarrollado en 2022 por un grupo de investigadores de Google Research y la Universidad de Boston. La misión de esta tecnología es la de poder entrenar a modelos de inteligencia artificial para personalizarlo según tus necesidades.



## ¿Como funciona? 🔩

El funcionamiento de esta técnica funciona en tres pasos.

1. En **primer lugar**, necesitas un modelo de difusión preentrenado, que es uno de esos sistemas de inteligencia artificial que pueden crear imágenes a partir de texto. Por ejemplo, se puede usar:
  - Stable Diffusion
  - DALL-E
  - Midjourney

  siempre y cuando funcionen con el proceso de ruido y denoising.

  Lo que hace esta técnica es crear una imagen completamente ruidosa, y luego ir quitando ese ruido reconstruyendo en el proceso una imagen totalmente original que se parezca a lo que le has pedido por texto. Pues es en este punto en el que Dreambooth ayudará con un modelo entrenado para que puedas obtener imágenes de sujetos concretos.

2.  el **segundo paso**, en el que necesitas un conjunto de imágenes del sujeto con el que quieres personalizar la IA. Puede ser un estilo, una cara, o lo que sea. Se recomienda tener un set de unas 8 o 10 imágenes como mínimo para poder entrenar el modelo.

  Entonces, lo que hace Dreambooth es **utilizar este set de imágenes para entrenar al modelo de difusión**, entrenar a la IA para que sepa reconocer lo que hay en ellas. Puede reconocer tu cara para luego poder dibujarla desde cero, así como un estilo o una posición.

3. Una vez has usado Dreambooth para entrenar a la IA, este sistema usará las imágenes del sujeto como punto de partida para el proceso de crear la imagen aleatoria, permitiendo que la IA tenga más información sobre cómo es el sujeto que quieres dibujar, y que así pueda hacer imágenes que se parezcan a él.

![Imagen](https://i.blogs.es/a42cbd/como-funciona/1366_2000.png)

# **Usando *Py-Dreambooth* en Jupyter Notebook Local**🧑‍💻

**Objectius:**

* Utilitza *Py-Dreambooth* per crear fàcilment imatges d'avatar d'IA a partir de fotos de tu, la teva família, amics o mascotes!

* Si us plau, executeu-vos en una màquina amb una GPU de 16GB o més.


## Instalació de paquets
* Instal·leu el paquet de Python *Py-Dreambooth* tal com es mostra a continuació.

In [ ]:
!pip install -q py_dreambooth

## Importa mòduls
* Hi ha diversos tipus de classes de model, però estaràs utilitzant el model més bàsic, el model Stable Diffusion Dreambooth `SDDreamboothModel`, però no t'has de preocupar per això ara mateix. 🤷‍♂️

In [ ]:
from py_dreambooth.dataset import LocalDataset
from py_dreambooth.model import SdDreamboothModel
from py_dreambooth.predictor import LocalPredictor
from py_dreambooth.trainer import LocalTrainer
from py_dreambooth.utils.image_helpers import display_images
from py_dreambooth.utils.prompt_helpers import make_prompt

Montem la relació entre el google drive i el quadern de jupyter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preparem les dades 📸

In [ ]:
DATA_DIR = "/content/drive/MyDrive/ANN/DREAMBOOTH/data"  # el directori amb fotos per a que el model s'entreni
OUTPUT_DIR = "/content/drive/MyDrive/ANN/DREAMBOOTH/models"  # El directori on s'ubicaran els fitxers de model entrenats

dataset = LocalDataset(DATA_DIR)

* Molt important! En el `DATA_DIR` definit anteriorment, posar les imatges (jpg o png) del subjecte que es vol entrenar.

* Per a aquesta tasca, necessitareu entre 10 i 20 solos, selfies d'alta qualitat preses amb diferents fons, il·luminació i expressions facials. Crec que es pot trobar un gran exemple al [repositori de GitHub de Joe Penna](https://github.com/JoePenna/Dreambooth-Stable-Diffusion).

![Samples](https://raw.githubusercontent.com/JoePenna/Dreambooth-Stable-Diffusion/refs/heads/main/readme-images/better-training-images.png)

* Utilitzeu el mètode de processament d'imatges següent per retallar les imatges en un quadrat centrat a la cara. Si el subjecte que el model està tractant d'aprendre no és una persona (per exemple, un gos), estableix l'argument `detect_face` argumentant com a  `False`.


In [ ]:
dataset = dataset.preprocess_images(detect_face=True)

A total of 8 images were found.


 38%|███▊      | 3/8 [00:00<00:00,  6.17it/s]

No faces detected in the image '443008034_395930086752782_7217331932050061307_n.jpg'.
No faces detected in the image '440173417_1436258973657135_9081022692963550822_n.jpg'.


 75%|███████▌  | 6/8 [00:00<00:00,  6.41it/s]

No faces detected in the image '429164819_452647503759342_2302826312178258320_n.jpg'.


100%|██████████| 8/8 [00:01<00:00,  5.50it/s]

A total of 5 images were preprocessed and stored in the path '/content/drive/MyDrive/ANN/DREAMBOOTH/data_preproc'.


## Entrena el model 🤖

* Ara és el moment d'entrenar el model! Digues al model el nom del subjecte al qual vols entrenar (p. ex., Joe) i la classe a la qual pertany.

* En definir un model, un dels arguments importants és quantes iteracions entrenar, o `max.train.steps`. S'accepta generalment que 800 a 1200 passos són apropiats per a una persona, i 200 a 400 passos són apropiats per a un animal no humà. El valor per defecte és 100 vegades el nombre de fotos que teniu. No cal que us preocupeu per això ara mateix ,🤷‍♂️, però si no us agraden els resultats de la imatge generada a continuació, aquest és el primer paràmetre a ajustar.


In [ ]:
SUBJECT_NAME = "mire"  # The name of the subject you want to learn
CLASS_NAME = "person"  # The class to which the subject you want to learn belongs

model = SdDreamboothModel(
    subject_name = SUBJECT_NAME,
    class_name = CLASS_NAME,
    max_train_steps=400,
)

trainer = LocalTrainer(output_dir = OUTPUT_DIR)

* El temps d'entrenament del model pot ser tan curt com unes poques desenes de minuts o com diverses hores.

In [ ]:
%%time
predictor = trainer.fit(model, dataset)

The model training has begun.
'max_train_steps' is set to 400.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1335 in time            │
│                                                                                                  │
│   1332 │   │   else:                                                                             │
│   1333 │   │   │   st = clock2()                                                                 │
│   1334 │   │   │   try:                                                                          │
│ ❱ 1335 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1336 │   │   │   │   out=None                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/py_dreambooth/trainer.py:124 in fit                      │
│                                                                                                  │
│   121 │   │   │   f"The model training has begun.\n'max_train_steps' is set to {max_train_step   │
│   122 │   │   │   self.logger,                                                                   │
│   123 │   │   )                                                                                  │
│ ❱ 124 │   │   _ = subprocess.run(shlex.split(command), check=True)                               │
│   125 │   │   log_or_print("The model training has ended.", self.logger)                         │
│   126 │   │                                                                                      │
│   127 │   │   predictor = LocalPredictor(model, self.output_dir, self.logger)                    │
│                                                                                                  │
│ /usr/lib/python3.10/subprocess.py:526 in run                                                     │
│                                                                                                  │
│    523 │   │   │   raise                                                                         │
│    524 │   │   retcode = process.poll()                                                          │
│    525 │   │   if check and retcode:                                                             │
│ ❱  526 │   │   │   raise CalledProcessError(retcode, process.args,                               │
│    527 │   │   │   │   │   │   │   │   │    output=stdout, stderr=stderr)                        │
│    528 │   return CompletedProcess(process.args, retcode, stdout, stderr)                        │
│    529                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
CalledProcessError: Command '['accelerate', 'launch', 
'/usr/local/lib/python3.10/dist-packages/py_dreambooth/scripts/train/train_sd_dreambooth.py', 
'--pretrained_model_name_or_path', 'stabilityai/stable-diffusion-2-1-base', '--instance_data_dir', 
'/content/drive/MyDrive/ANN/DREAMBOOTH/data_preproc', '--instance_prompt', 'a photo of mire person', 
'--num_class_images', '200', '--output_dir', '/content/drive/MyDrive/ANN/DREAMBOOTH/models', '--resolution', '768',
'--train_batch_size', '1', '--learning_rate', '2e-06', '--lr_scheduler', 'constant', '--lr_warmup_steps', '0', 
'--validation_prompt', 'a photo of mire person with Eiffel Tower in the background', '--compress_output', 'False', 
'--with_prior_preservation', 'True', '--prior_loss_weight', '1.0', '--cl

## Bibliografia 💃
* [Stable Diffusion](https://www.xataka.com/basics/como-entrenar-a-ia-tu-cara-para-crear-avatares-gratis-stable-diffusion)
* [DreamBooth](https://www.xataka.com/basics/dreambooth-que-como-funciona-esta-tecnologia-para-entrenar-a-inteligencia-artificial)
* [Py-Dreambooth](https://github.com/JoePenna/Dreambooth-Stable-Diffusion?tab=readme-ov-file)